In [ ]:
# Must precede any llm module imports
from langtrace_python_sdk import langtrace
langtrace.init(api_key = '8e0dafdc118df1613b10dbdda776b0b062427b0a9b1cb18b719688714e1ea445',
  disable_tracing_for_functions= {
    "open_ai": [ # All supported functions for openai
      'openai.embeddings.create',
    ]
  },
  disable_instrumentations={"all_except": ['openai']}
)

Initializing Langtrace SDK..
⭐ Leave our github a star to stay on top of our updates - https://github.com/Scale3-Labs/langtrace
Skipping openai due to error while instrumenting: No module named 'openai.resources.responses'
Exporting spans to Extraction..
Langtrace Project URL: https://app.langtrace.ai/project/cmf58kthl000f5i55fmxndtpw/traces


/Users/pastalover/miniconda3/envs/sisyphus_context/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import json
from ast import literal_eval
from typing import Optional, Literal
from functools import partial

import dspy
from pydantic import BaseModel, Field, field_validator
from langchain_openai import ChatOpenAI

from sisyphus.heas.label import label_paras
from sisyphus.chain.paragraph import Paragraph, ParagraphExtend
from sisyphus.chain import Filter, Writer
from sisyphus.strategy.run_strategy import ExtractStrategy
from sisyphus.strategy.pydantic_models_general import Processing, Material, MaterialDescriptionBase
from sisyphus.strategy.utils import get_paras_with_props, get_synthesis_paras
from sisyphus.heas.prompt import *
from sisyphus.utils.helper_functions import get_plain_articledb, get_create_resultdb
from sisyphus.heas.synthesis import get_synthesis_prompt
from sisyphus.strategy.llm_models import categorize_agent


lm = dspy.LM('openai/gpt-4.1-mini')
dspy.configure(lm=lm)
chat_model = ChatOpenAI(model='gpt-4.1-mini')

class Strength(BaseModel):
    """Tensile/Compressive test results"""
    ys: Optional[str] = Field(description="Yield strength with unit")
    uts: Optional[str] = Field(description="Ultimate tensile/compressive strength with unit")
    strain: Optional[str] = Field(description="Fracture strain. If in percentage form, please add '%' sign, else return as decimal form. Example: 0.5 or 50%")
    temperature: Optional[str] = Field(description="Test temperature with unit, if not specified, return 'room temperature'")
    strain_rate: Optional[str] = Field(description="Strain rate with unit")
    test_type: Literal['tensile', 'compressive']


class Phase(BaseModel):
    """Phase information"""
    phases: list[str] = Field(description="List of phases present in the material")

class Processing(BaseModel):
    """Processing route for a material
    Return field steps as '[]' if you cannot find any. For fields with unknown value, filled with empty string"""
    steps: str = Field(description="""List of processing steps in chronological order, form them as json object. For example: [{"induction melting": {"temperature": "1500 K"}}, {"annealed": {"temperature": "800 K", "duration": "1h"}}]""")

    @field_validator('steps', mode='after')
    @classmethod
    def load(cls, value: str):
        try:
            value = json.loads(value)
        except:
            value = literal_eval(value)
        return value

prompt_config = \
    {
        'strength': (EXTRACT_PROPERTY_SYS_GENERIC_PROMPT, STRENGTH_PROMPT),
        'phase': (EXTRACT_PROPERTY_SYS_GENERIC_PROMPT, PHASE_PROMPT),
        'synthesis': (EXTRACT_PROCESS_SYS_GENERIC_PROMPT, PROCESS_PROMPT)
    }

def reconstr_c(paragraphs):
    p_exp = get_synthesis_paras(paragraphs) + get_paras_with_props(paragraphs, 'composition')
    p_str_paras = get_paras_with_props(paragraphs, 'strength') + get_paras_with_props(paragraphs, 'strain_rate')
    p_phase_paras = get_paras_with_props(paragraphs, 'phase')
    # Include synthesis paragraphs for context if available
    p_str = ParagraphExtend.from_paragraphs(p_str_paras + p_exp, type='strength') if p_str_paras else None
    p_phase = ParagraphExtend.from_paragraphs(p_phase_paras + p_exp, type='phase') if p_phase_paras else None
    p_exp_final = ParagraphExtend.from_paragraphs(p_exp, type='synthesis') if p_exp else None
    return {
        "strength": p_str,
        "phase": p_phase,
        "synthesis": p_exp_final
    }

models_d = {
    'strength': Strength,
    'phase': Phase,
    'synthesis': Processing
}
 


/Users/pastalover/miniconda3/envs/sisyphus_context/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from typing import Literal
from sisyphus.utils.helper_functions import get_title_abs, render_docs

class ClassifyPaper(dspy.Signature):
    """assign label to HEAs (high entropy alloys) paper based on their title and abstract."""
    context: str = dspy.InputField(desc='Title and abstract of the paper')
    label: Literal['hea_experimental', 'hea_theoretical', 'irrelevant'] = dspy.OutputField(desc="Pay attention to keywords such as 'molecular dynamics' or 'machine learning,' which should be labeled as hea_theoretical. Label keywords related to fabrication processes as hea_experimental.")
    mechanical_relevancy: bool = dspy.OutputField(desc='whether this paper describe the mechanical properties such as tensile or compressive')
classifier_paper = dspy.ChainOfThought(signature=ClassifyPaper)

def paper_filter(docs):
    title, abstract = get_title_abs(docs)
    prediction = classifier_paper(context=render_docs(abstract, title))
    if prediction.label == 'hea_experimental' and prediction.mechanical_relevancy:
        return docs
    return

In [3]:
es = ExtractStrategy(
    reconstruct_paragraph_context_func=reconstr_c,
    formatted_func=get_synthesis_prompt,
    pydantic_models_dict=models_d,
    save_to='op.jsonl'
)
es.build(prompt_config=prompt_config, chat_model=chat_model)
db = get_plain_articledb('heas_1531')
getter = Filter(db)
result_db = get_create_resultdb('context_isolated')
writer = Writer(result_db)

/Users/pastalover/miniconda3/envs/sisyphus_context/lib/python3.10/site-packages/pydantic/main.py:1552: RuntimeWarning: fields may not start with an underscore, ignoring "__tablename__"
  warnings.warn(f'fields may not start with an underscore, ignoring "{f_name}"', RuntimeWarning)


In [4]:
from sisyphus.chain.chain_elements import run_chains_with_extarction_history_multi_threads

In [ ]:
# LABEL
labeled_database = get_plain_articledb('heas_labeled_extend')
labeled_database.create_db()
def save(paras):
    labeled_database.dump_state(paras)
label_chain = getter + paper_filter + label_paras + save
# run_chains_with_extarction_history_multi_threads(label_chain, 'heas_test', 5, 'heas_labeled_extend')

100%|██████████| 20/20 [00:13<00:00,  1.43it/s]


In [7]:
result_db.clear_tables()

In [8]:
# TEST
labeled_getter = Filter(labeled_database)
def load(docs):
    return [Paragraph.from_labeled_document(doc, id_) for id_, doc in enumerate(docs)]
extract_chain = labeled_getter + load + es + writer
extract_chain.compose('10.1002&sol;adem.201900587.html')

In [8]:
get_paras_chain = labeled_getter + load
paras = get_paras_chain.compose('10.1002&sol;adem.201900587.html')
ps = get_paras_with_props(paras, 'phase')
for p in ps:
    print(p.id)

9
11


In [ ]:
# chain = getter + paper_filter + label_paras + es + writer
# # run_chains_with_extarction_history_multi_threads(chain, 'heas_test', 5, 'context_isolated', 5)
# chain.compose('10.1002&sol;adem.201900587.html')
# # chain.compose('10.1016&sol;j.ijrmhm.2023.106163.html')